In [3]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS,analyzer='word', binary=True, min_df=10, max_df=.04)
vectorizer.fit(newsgroups_train.data)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.04, max_features=None, min_df=10,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=frozenset({'where', 'hers', 'give', 'sometime', 'please', 'being', 'whose', 'all', 'since', 'yourself', 'seeming', 'done', 'hereafter', 'last', 'formerly', 'co', 'onto', 'cant', 'ours', 'cry', 'thereupon', 'seemed', 'or', 'toward', 'front', 'back', 'will', 'at', 'was', 'after', 'which', '...once', 'both', 'therein', 'yours', 'least', 'thick', 'only', 'keep', 'get', 'many', 'while', 'has'}),
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [10]:
X_train = vectorizer.fit_transform(newsgroups_train.data)
X_train.shape

(11314, 10299)

In [15]:
from tqdm import tqdm

def fun(X, number, a, b, n_iter=10):

    m1 = np.zeros((number, X.shape[1]))
    m2 = np.zeros((X.shape[0], number))
    m3 = np.zeros(number)
    
    docs, words = X.nonzero()
    z = np.random.choice(number, len(docs))
    
    for doc, word, z_ in zip(docs, words, z):
        m2[doc, z_] += 1
        m1[z_, word] += 1
        m3[z_] += 1
    
    for iter_ in tqdm(range(n_iter)):
        for i in range(len(docs)):
            m2[docs[i], z[i]] -= 1
            m1[z[i], words[i]] -= 1
            m3[z[i]] -= 1
            
            param = (m2[docs[i], :] + a) * (m1[:, words[i]] + b[words[i]]) / \
                (m3 + b.sum())
            z[i] = np.random.choice(np.arange(number), p=param / param.sum())
            
            m2[docs[i], z[i]] += 1
            m1[z[i], words[i]] += 1
            m3[z[i]] += 1
    
    return z, m1, m2, m3

In [18]:
number = 20
z, m1, m2, m3 = fun(X_train, number, 1 * np.ones(number), 1 * np.ones(X_train.shape[1]), 60)



  0%|          | 0/60 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
top_words = np.argsort(m1, axis=1)[:, :-11:-1]

for topic in range(20):
    doc = np.zeros((1, X_train.shape[1]))
    for word in top_words[topic]:
        doc[0, word] = 1
    print(' Tag {}:\t{}'.format(topic, '\t'.join(vectorizer.inverse_transform(doc)[0])))

In [ ]:
newsgroups_train.target_names